In [79]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.DataIO import DataIO
from skopt.space import Real, Integer, Categorical
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender,MultiThreadSLIM_SLIMElasticNetRecommender



columns_name = ['user_id','item_id','impression_list','data']

dataset_l = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_length.csv', sep=',')
dataset_t = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_type.csv', sep=',')
dataset = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/interactions_and_impressions.csv', sep=',')
test_users = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_target_users_test.csv',sep = ',')
dataset.columns = columns_name


c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [63]:
df = dataset_l[dataset_l.data < 50000]

In [64]:
data = df.data.values

In [65]:
dataPR = data.reshape(-1,1)

In [66]:
dataPR

array([[   1],
       [   1],
       [  21],
       ...,
       [   1],
       [1001],
       [   1]], dtype=int64)

In [67]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=7, random_state=0).fit(dataPR)
print(kmeans.labels_)
print(len(kmeans.labels_))
print(kmeans.cluster_centers_)

[1 1 6 ... 1 0 1]
19630
[[1.00295652e+03]
 [2.68736182e+00]
 [1.00000000e+04]
 [3.76028571e+02]
 [1.53320000e+03]
 [1.25692708e+02]
 [2.78938462e+01]]


In [68]:
data = df.data.values
dataPR = data.reshape(-1,1)
kmeans = KMeans(n_clusters=7, random_state=0).fit(dataPR)
df['data'] = kmeans.labels_.copy()

In [88]:
import scipy.sparse as sps
def build_ICM_lengh_kmeans(dataset_l):
    df_l = dataset_l.copy()
    df_l.drop(columns = ['feature_id'], inplace = True)
    df = dataset_l[dataset_l.item_id <= 24506]
    data = df.data.values
    dataPR = data.reshape(-1,1)
    kmeans = KMeans(n_clusters=7, random_state=0).fit(dataPR)
    df.data = kmeans.labels_.copy()
    num_items, min_item_id, max_item_id = len(df.data.unique()), df.data.min(), df.data.max()
    item = range(0,24506)
    df_1 = pd.get_dummies(df.data)
    column = range(num_items)
    df_1.columns = column
    df_1.index = df.item_id
    ICM_lenght = pd.DataFrame(np.zeros((24507,7)))
    ICM_lenght[ICM_lenght.index.isin(df_1.index)] = df_1
    return sps.csr_matrix(ICM_lenght)

In [87]:
build_ICM_lengh_kmeans(dataset_l)

c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


<24507x7 sparse matrix of type '<class 'numpy.float64'>'
	with 19630 stored elements in Compressed Sparse Row format>